# Algoritmo que permite resolver problemas QUBO densos empleando Tensor Network

En este notebook vamos a crear e implementar un algoritmo en tensor networks basado en el método de las señales y evolución en tiempo imaginario para resolver problemas QUBO. Utiliza un método de eliminación de capas de evolución para reducir la complejidad computacional.

Versiones:

- alpha 0: Implementación básica.
- alpha 1: Añade el método de eliminación de capas.
- alpha 2: Versión que va comprimiendo el MPS de derecha a izquierda
- alpha 3: version que aplica el problema QUBO a los vecinos cercanos, la idea de esta versión es aplicar matrices recursivamente, además de intentar aprovechar los cálculos intermedios

In [2]:
# Librerias
import numpy as np
import torch
from quantum_sim.main.general_functions import dinariy_list
from quantum_sim.TensorNetwork.QUBO.qubo_core.qubo_solvers import qubo_dimod_solver, recocido_simulado, qubo_solver_rs, random_qubo_solver
from quantum_sim.TensorNetwork.QUBO.qubo_core.qubo_auxiliar_functions import matrix_QUBO_to_dict, evaluar_qubo, generar_matriz_qubo
from quantum_sim.main.guardar_experimentos import plot_function
from itertools import product


%load_ext autoreload
%autoreload 2

0005
0054


---
# Funciones de la tensor network

## Definimos los nodos de la Tensor Network

In [3]:
from itertools import combinations
from operator import index

from numpy import dtype
from sklearn import neighbors


def node_0(Q_matrix_0: float, dits: int, tau: float):
    tensor = np.zeros((dits, dits))
    for index_a in range(dits):
        tensor[index_a, index_a] = np.exp(-tau* Q_matrix_0 * index_a)
    return tensor

def node_grow(Q_matrix_row: np.array, dits: int, num_neight_updown, tau: float):
    size_1 = dits**num_neight_updown
    size_2 = size_1 * dits
    tensor = np.zeros((size_1, size_2))
    dit_list = list(range(dits))
    combinations_up = product(dit_list, repeat = num_neight_updown)
    for elemento in combinations_up:
        index_up = 0 
        if num_neight_updown != 1:
            for aux in range(num_neight_updown):
                index_up += dits**aux*elemento[aux]
            for last_index in range(dits):
                index_down = index_up + dits**(aux+1)* last_index
                tensor[index_up, index_down] = 1
                for aux in range(len(elemento)):
                    tensor[index_up, index_down] *= np.exp(-tau * Q_matrix_row[aux] * last_index * elemento[aux])
                tensor[index_up, index_down] *= np.exp(-tau * Q_matrix_row[-1] * last_index **2)
        else:
            index_up = elemento
            for last_index in range(dits):
                index_down = int(elemento[0]) + last_index*dits
                tensor[index_up, index_down] = 1
                for aux in range(len(elemento)):
                    tensor[index_up, index_down] *= np.exp(-tau * Q_matrix_row[aux] * last_index * elemento[aux])
                tensor[index_up, index_down] *= np.exp(-tau * Q_matrix_row[-1] * last_index **2)

    return tensor

def node_intermediate(Q_matrix_row: np.array, dits: int, num_neight_updown, tau: float):
    size_1 = dits**num_neight_updown
    tensor = np.zeros((size_1, size_1))
    dit_list = list(range(dits))
    combinations_up = product(dit_list, repeat = num_neight_updown)
    for elemento in combinations_up:
        if num_neight_updown != 1:
            index_up = 0 
            for aux in range(num_neight_updown):
                index_up += dits**aux*elemento[aux]
            for last_index in range(dits):
                index_down = 0
                for aux in range(1,num_neight_updown):
                    index_down += dits**(aux-1)*elemento[aux]
                index_down += dits**(aux)*last_index

                tensor[index_up, index_down] = 1
                for aux in range(1,len(elemento)+1):
                    tensor[index_up, index_down] *= np.exp(-tau * Q_matrix_row[aux-1]*last_index*elemento[aux-1])
                tensor[index_up, index_down] *= np.exp(-tau * Q_matrix_row[-1]*last_index**2)
        else:
            for last_index in range(dits):
                tensor[elemento, last_index] = np.exp(-tau * Q_matrix_row[-1]*last_index**2 + Q_matrix_row[-2]*last_index*elemento[0])

    return tensor
                    

def last_tensor(Q_matrix_column_input: np.array, dits: int, tau:float):
    n_neighbors = (len(Q_matrix_column_input)-1)
    tensor_size = int(dits**(n_neighbors))
    tensor = np.zeros((tensor_size))
    dit_list = list(range(dits))
    combinations_up = product(dit_list, repeat = n_neighbors)
    for element in combinations_up:
        index_up = 0
        for aux in range(n_neighbors):
            index_up += dits**aux*element[aux]
        for last_index in range(dits):
            tensor_aux =1
            for el in range(len(element)):
                tensor_aux *= np.exp(-tau*Q_matrix_column_input[el]*element[el]*last_index)
            tensor_aux *= np.exp(-tau*Q_matrix_column_input[-1]*last_index**2)
            tensor[index_up] += tensor_aux
    return tensor



def new_inital_tensor(Q_matrix_row, dits: int, size_2, solution, n_neigh, tau:float):
    size_1 = dits
    tensor = np.zeros((size_1, size_2))
    dit_list = list(range(dits))
    
    combinations_up = product(dit_list, repeat = n_neigh-1)
    index_down = 0

    solution = tuple(solution[-n_neigh+2:])
    for aux in range(len(solution)):
        index_down += dits**aux*solution[aux]

    for element in combinations_up:
       
        if element[:-1] == solution:

            index_down_aux = index_down + dits**(n_neigh-2)*element[-1]
            tensor[element[-1], index_down_aux] = 1
             
            for el in range(len(element)):          
                tensor[element[-1], index_down_aux] *= np.exp(-tau*Q_matrix_row[el]*element[el]*element[-1])
            #tensor[element[-1], index_down_aux] *= np.exp(tau*Q_matrix_row[-1]*element[-1]**2)
    return tensor



## Generación de la Tensor Network

In [4]:
def tensor_network_generator(Q_matrix:np.array, dits:float, n_neighbors:int, tau: float):
    """   
    Args:

    Return:

    """
    
    n_variables  = len(Q_matrix[0])
    intermediate_tensors = []
    # generation of the first node
    tensor = node_0(Q_matrix[0][0], dits, tau)
    intermediate_tensors.append(tensor)
    
    # Generation of the intermediate nodes
    for variable in range(1, n_variables-1):

        Q_matrix_row_input = Q_matrix[variable][max(0, variable-2):variable+1]
        if variable < n_neighbors - 1:
            tensor = node_grow(Q_matrix_row_input, dits, variable, tau)
            intermediate_tensors.append(tensor)
        else:
            tensor = node_intermediate(Q_matrix_row_input, dits, n_neighbors - 1, tau)
            intermediate_tensors.append(tensor)
    Q_matrix_row_input = Q_matrix[-1][-n_neighbors:]
    tensor = last_tensor(Q_matrix_row_input, dits, tau)
    intermediate_tensors.append(tensor)

    return  intermediate_tensors

## Conexión y contracción de toda la Tensor Network

In [5]:
def tensor_network_contraction(tensor_list: list):
    n_tensores = len(tensor_list)
    intermediate_tensors = []
    tensor = tensor_list[-1]
    intermediate_tensors.append(tensor)
    for index_ in range(n_tensores -1, 0, -1):
        current_tensor = tensor_list[index_ -1]
        tensor = current_tensor @ tensor
        intermediate_tensors.append(tensor)
        if np.max(current_tensor)>1e200:
            current_tensor*=1e-30
    intermediate_tensors.reverse()
    return tensor, intermediate_tensors
    

---
# Función general

Esta es la función que se encarga del proceso general. Se encarga del proceso de minimización resolviendo iterativamente cada una de las variables. Su proceso consiste en la creación de la tensor network, su contracción y la determinación de la variable a partir del vector resultante.

In [6]:
sol = np.zeros((2))
print(len(sol))

2


In [71]:
from unittest import result

from quantum_sim.main.general_functions import base_a_decimal


def qubo_solver(Q_matrix:np.array, tau:float, dits: int, n_neighbors: int)->np.array:
    n_variables = Q_matrix.shape[0]
    #print(Q_matrix)
    solution = np.zeros(n_variables, dtype = int)
    tensor_network = tensor_network_generator(Q_matrix, dits, n_neighbors, tau)

    result_contraction, intermediate_tensors = tensor_network_contraction(tensor_network)
    solution[0] = np.argmax(abs(result_contraction))
    for i in range(2,len(Q_matrix)):  # Iteramos sobre las filas
        Q_matrix[i, i] += Q_matrix[i, 0] * solution[0]


    for node in range(1, n_variables-1):
        #print("QMATRX Grande")
        #print(Q_matrix)
        Q_matrix_row = Q_matrix[node][node-1:node+1]
       
        new_tensor = new_inital_tensor(Q_matrix_row, dits, intermediate_tensors[node + 1].shape[0], solution[:node], n_neighbors, tau)
        print("new_t")
        print(new_tensor)
        solution[node] = np.argmax(abs(new_tensor @ intermediate_tensors[node + 1]))

        for i in range(node + 2,len(Q_matrix)):  # Iteramos sobre las filas
            Q_matrix[i, i] += Q_matrix[i, node] * solution[node]
        print("Q_m", Q_matrix)


    cost1 = evaluar_qubo(Q_matrix, solution)
    solution[-1] = 1
    cost2 = evaluar_qubo(Q_matrix, solution)
    if cost1 < cost2:
        solution[-1] = 0
    return solution


---
# Pruebas

In [72]:
n_variables = 5
n_vecinos = 2
dits = 2
tau = 100
np.random.seed(20)
# Generamos el caso

Q_matrix = generar_matriz_qubo(n_variables, n_vecinos + 1)
Q_matrix/= np.linalg.norm(Q_matrix)

Q_matrix_dict = matrix_QUBO_to_dict(Q_matrix)

Q_matrix_copy = Q_matrix.copy()
# Inicial RS
x_inicial = np.random.randint(2, size=n_variables)
'''
solution = recocido_simulado(Q_matrix, x_inicial, 10.0, 0.99, int(1e4))
#print('Solution RS:     ', str(solution))
print('Coste de RS:     ', evaluar_qubo(Q_matrix, solution))
'''
'''
# Dimod
solution_dimod = qubo_dimod_solver(Q_matrix_dict, "neal")
print('Coste de dimod iter: ', evaluar_qubo(Q_matrix, solution_dimod))
'''
best_c = random_qubo_solver(Q_matrix)
print('Coste de Random: ', evaluar_qubo(Q_matrix, best_c))

# TN
solution = qubo_solver(Q_matrix_copy, tau, dits, n_vecinos + 1)
print('Solution TN:     ', str(solution))
print('Coste de TN:     ', evaluar_qubo(Q_matrix, solution))


# Dimod
solution_dimod = qubo_dimod_solver(Q_matrix_dict, "neal")
print('Coste de dimod iter: ', evaluar_qubo(Q_matrix, solution_dimod))
'''
# RS
mejor_solucion, mejor_valor = recocido_simulado_qudo(Q_matrix, [0, 1, 2], 100, 0.95, 1000)
print("Mejor solución encontrada:", mejor_solucion)
print("Mejor valor objetivo:", evaluar_qubo(Q_matrix, mejor_solucion))
'''


Coste de Random:  -0.062489460520388784
new_t
[[1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 4.89038778e-07 0.00000000e+00]]
Q_m [[ 0.10555746  0.          0.          0.          0.        ]
 [ 0.22967502  0.14530824  0.          0.          0.        ]
 [-0.27270491  0.26183226  0.3389637   0.          0.        ]
 [ 0.         -0.45909983  0.30096802 -0.14675202  0.        ]
 [ 0.          0.          0.07388459 -0.38473623  0.32369055]]
new_t
[[0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 8.08654313e-27]]
Q_m [[ 0.10555746  0.          0.          0.          0.        ]
 [ 0.22967502  0.14530824  0.          0.          0.        ]
 [-0.27270491  0.26183226  0.3389637   0.          0.        ]
 [ 0.         -0.45909983  0.30096802 -0.14675202  0.        ]
 [ 0.          0.          0.07388459 -0.38473623  0.32369055]]
new_t
[[1.00000000e+00 0.00000000e+00 0.00000000e+00 0

'\n# RS\nmejor_solucion, mejor_valor = recocido_simulado_qudo(Q_matrix, [0, 1, 2], 100, 0.95, 1000)\nprint("Mejor solución encontrada:", mejor_solucion)\nprint("Mejor valor objetivo:", evaluar_qubo(Q_matrix, mejor_solucion))\n'